In [ ]:
# Import libraries and set desired options
import pickle
from pathlib2 import Path
import numpy as np
import pandas as pd
from datetime import datetime
import os

from scipy.sparse import csr_matrix, hstack, vstack, coo_matrix
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

import seaborn as sns

from hyperopt import hp, tpe, fmin, STATUS_OK

from catboost import CatBoostRegressor